In [79]:
#loads review data
import pandas as pd
from scipy import sparse
from collections import defaultdict
from scipy import sparse
from sklearn import model_selection as cv
import numpy as np

header = ["reviewerId","productId","rating","time"]
df = pd.read_csv('ratings_Video_Games.csv', names=header)
#df = pd.read_csv('item_dedup.csv', names=header)
n_users = df.reviewerId.unique().shape[0]
n_items = df.productId.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of items = ' + str(n_items))


train_data, test_data = cv.train_test_split(df, test_size=0)

# Create dict for product id 
#df.productId.unique()
productDict = defaultdict(dict)
userDict = defaultdict(dict)
count = 0


for id in train_data.reviewerId.unique():
    userDict[id] = count
    count += 1

count = 0
for id in train_data.productId.unique():
    productDict[id] = count
    count += 1
    
n_users = len(userDict)
n_items = len(productDict)

print(n_users, " ",  n_items,"\n")

mat = sparse.lil_matrix((n_users, n_items),dtype=np.float32)
        
print(productDict['B00004XOLO'])
print(userDict['A2EHIE51TERF2C'])


for line in train_data.itertuples():
    mat[userDict[line[1]],productDict[line[2]]] = line[3]

print(mat.getnnz)
print("done")

Number of users = 826767 | Number of items = 50210
826767   50210 

8642
690401
<bound method lil_matrix.getnnz of <826767x50210 sparse matrix of type '<class 'numpy.float32'>'
	with 1324753 stored elements in LInked List format>>
done


In [80]:
#Joseph and Jack
#loading item metadata
import pandas as pd
import gzip

def parse(path):
    print("parsing")
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    print("getting df")
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

meta_df = getDF('meta_Video_Games.json.gz')
print("done: meta_df holds all item metadata")

getting df
parsing
done: meta_df holds all item metadata


In [81]:
#Joseph and Jack
#this code cell is just used to play around with accessing the data.
#this cell does not do anything important. 
print( meta_df.iloc[0], "\n_______________\n") #printing the first element of the df
singleItem = meta_df.loc[meta_df['asin'] == 'B000066TS5']
print(singleItem,"\n_______________\n" ) #printing the info of the item with the specified item ID
for item in singleItem.iterrows():
    print(item[1][4], "\n_______________\n") #['also_bought']
    print(item[1][4]['also_bought'], "\n_______________\n")
    print(item[1][4]['also_bought'][0], "\n_______________\n")
    print(item[1][5], "\n_______________\n") #ranking
    print(item[1][6], "\n_______________\n") #list of category lists
    print(item[1][2], "\n_______________\n") #price

print( "\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

"""print( meta_df.iloc[0], "\n_______________\n") #printing the first element of the df
singleItem2 = meta_df.loc[meta_df['asin'] == 'B0001I1LD6']
print(singleItem2,"\n_______________\n" ) #printing the info of the item with the specified item ID
for item in singleItem2.iterrows():
    print(item[1][4], "\n_______________\n")
    if "also_viewed" in item[1][4]:
        print( "THIS WORKS!!!!!!!!!!!!!")
    print(item[1][6], "\n_______________\n")"""

asin                                                  0078764343
description                                    Brand new sealed!
price                                                      37.98
imUrl          http://ecx.images-amazon.com/images/I/513h6dPb...
related        {'also_bought': ['B000TI836G', 'B003Q53VZC', '...
salesRank                                 {'Video Games': 28655}
categories                      [[Video Games, Xbox 360, Games]]
title                                                        NaN
brand                                                        NaN
Name: 0, dtype: object 
_______________

            asin                                        description  price  \
7599  B000066TS5  Ever wondered what would happen if the worlds ...   52.5   

                                                  imUrl  \
7599  http://ecx.images-amazon.com/images/I/51MtbYNR...   

                                                related             salesRank  \
7599  {'also_boug

'print( meta_df.iloc[0], "\n_______________\n") #printing the first element of the df\nsingleItem2 = meta_df.loc[meta_df[\'asin\'] == \'B0001I1LD6\']\nprint(singleItem2,"\n_______________\n" ) #printing the info of the item with the specified item ID\nfor item in singleItem2.iterrows():\n    print(item[1][4], "\n_______________\n")\n    if "also_viewed" in item[1][4]:\n        print( "THIS WORKS!!!!!!!!!!!!!")\n    print(item[1][6], "\n_______________\n")'

In [82]:
#Joseph and Jack
def getItemInfoVector(itemID): #counts the number of 1, 2, 3, 4, and 5 star ratings and returns an array of those totals (for a single ID)
    returnedVector = [0, 0, 0, 0, 0] #1,2,3,4,5 (so that if you want the number of n star ratings, you look at index n-1)
    reviewList = df.loc[df['productId'] == itemID]
    itemMeta = meta_df.loc[meta_df['asin'] == itemID]
    #print( reviewList, "\n_______\n")
    for item in reviewList.iterrows():
        itemRating = item[1][2]
        if( itemRating == 5):
            returnedVector[4]+=1
        elif( itemRating == 4):
            returnedVector[3]+=1
        elif( itemRating == 3):
            returnedVector[2]+=1
        elif( itemRating == 2):
            returnedVector[1]+=1
        elif( itemRating == 1):
            returnedVector[0]+=1
    #print( itemID, "'s ratings (1-5 stars): ",ratings)
    for item in itemMeta.iterrows():
        itemRank = 0
        itemPrice = 0
        try:
            itemRank = item[1][5]["Video Games"]
        except:
            pass
            #print("WARNING: NO ITEM RANK ###########")
            #print(item)
        try:
            itemPrice = item[1][2]
        except:
            pass
            #print("WARNING: NO ITEM PRICE $$$$$$$$$$$")
            #print(item)
        returnedVector.append(itemRank)
        returnedVector.append(itemPrice)
        
    return returnedVector
    #print(list(productDict.keys())[list(productDict.values()).index(itemId)])

print("done")

done


In [16]:
#Joseph and Jack
#getItemInfoVector("B000066TS5")
from scipy import spatial


def vectorCosineSimilarity( baseVector, otherVector):
    badVector = [0,0,0,0,0]
    if otherVector == badVector:
        return 0
    return 1-spatial.distance.cosine(baseVector, otherVector)

def findItemBFS( relatedItemID, depth, baseVector, explored): #returns a tuple with (cos_sim, itemID)
    bestItem = (0, relatedItemID) 
    if relatedItemID in explored: #dupeCheck
        return bestItem
    otherVector = getItemInfoVector(relatedItemID)
    bestItem =( vectorCosineSimilarity(baseVector, otherVector), relatedItemID)
    #print( bestItem)
    #depth check here
    if depth == 0:
        return bestItem, explored
    #afterwords, go through related items
    itemMetaData = meta_df.loc[meta_df['asin'] == relatedItemID]
    for metaItem in itemMetaData.iterrows(): #fake for loop with only one item
        if "also_bought" in metaItem[1][4]:
            for relatedItem in metaItem[1][4]['also_bought']: #real for loop to go through all related items
                #print( relatedItem )
                if relatedItem in explored:
                    continue
                bfsResult, explored = findItemBFS( relatedItem , depth-1, baseVector, explored)
                if bfsResult[0] > bestItem[0]:
                    bestItem = bfsResult
                if relatedItem not in explored:
                    explored.append( relatedItem )
        elif "also_viewed" in metaItem[1][4]:
            for relatedItem in metaItem[1][4]['also_viewed']: #real for loop to go through all related items
                #print( relatedItem )
                if relatedItem in explored:
                    continue
                bfsResult, explored = findItemBFS( relatedItem , depth-1, baseVector, explored)
                if bfsResult[0] > bestItem[0]:
                    bestItem = bfsResult
                if relatedItem not in explored:
                    explored.append( relatedItem )
        else:
            print( "WARNING: DID NOT FIND RELATED ITEMS IN THE FOLLOWING !!!!!!!!!!!!!!")
            print( metaItem[1][4])
    return bestItem, explored

def getMostSimilarItem( item, owned): #finds the item with the most similar cosine similarity
    depth = 1
    exploredList = owned #to keep track of visited items
    bestItem = (0, item[1][1]) #a tuple: (cos_sim, itemID)
    #print( item[1][1])
    mainVector = getItemInfoVector(item[1][1]) # the vector we are doing the cosine similarity with
    print( mainVector)
    mainItemMetaData = meta_df.loc[meta_df['asin'] == item[1][1]]
    for metaItem in mainItemMetaData.iterrows(): #fake for loop with only one item
        print("ITEM METADATA: \n", metaItem[1])
        #print(metaItem[1][0])
        #print(metaItem[1][4])
        if "also_bought" in metaItem[1][4]:
            for relatedItem in metaItem[1][4]['also_bought']: #real for loop (TODO: handle cases where also_bought is not there)
                #print( relatedItem )
                if relatedItem in exploredList:
                    continue
                bfsResult, exploredList = findItemBFS( relatedItem , depth, mainVector, exploredList)
                if bfsResult[0] > bestItem[0]:
                    bestItem = bfsResult
                if relatedItem not in exploredList:
                    exploredList.append( relatedItem )
        elif "also_viewed" in metaItem[1][4]:
            for relatedItem in metaItem[1][4]['also_viewed']: #real for loop (TODO: handle cases where also_bought is not there)
                #print( relatedItem )
                if relatedItem in exploredList:
                    continue
                bfsResult, exploredList = findItemBFS( relatedItem , depth, mainVector, exploredList)
                if bfsResult[0] > bestItem[0]:
                    bestItem = bfsResult
                if relatedItem not in exploredList:
                    exploredList.append( relatedItem )
        else:
            print( "WARNING: DID NOT FIND RELATED ITEMS IN THE FOLLOWING !!!!!!!!!!!!!!")
            print( metaItem[1][4])
    #print( item[1][1], " ", mainVector)
    bestItemMetaData = meta_df.loc[meta_df['asin'] == bestItem[1]]
    for metaItem in bestItemMetaData.iterrows():
        print("~~~~~~~~~~~~~~\nBEST ITEM'S METADATA:\n", metaItem[1])
    return bestItem
    
print("done")

done


In [77]:
#Joseph and Jack
import math
def dataSplit( reviewList): #find the median time and split the data based on that (first half for rec, second for checking)
    print( "splitting data")
    recData = []
    checkData = []
    #find the count of reviews in the reviewList
    n_reviews = reviewList.time.unique().shape[0]
    splitPoint = math.ceil(n_reviews/2)
    print("~~~~~",n_reviews," reviews~~~~~", " split into: ", splitPoint," and ",math.floor(n_reviews/2),"~~~~~")
    #split data so that the oldest half of the reviewList is used for the reccomendations
    #and the newest half is used for accuracy checking
    #before we do that, we should make sure the reviews are ordered chronologically
    reviewList = reviewList.sort_values(by=['time'])
    print( "reviews sorted by time: \n",reviewList)
    recData = reviewList[0:splitPoint]
    checkData = reviewList[splitPoint:]
    print( "-----------\nData used for recommedations:\n",recData,"\n----------\nData used for checking accuracy:\n", checkData,"\n---------")
    return recData, checkData

def recommendItems( userID ): #the main function (takes in a user ID)
    userReviewList = df.loc[df['reviewerId'] == userID]
    #print( "User's Item List: \n", userReviewList)
    recData, checkData = dataSplit( userReviewList)
    resultsList = []
    counter = 0
    print( "\nSearching for similar items...")
    owned = []
    for item in userReviewList.iterrows():
        owned.append(item[1][1])
    for item in userReviewList.iterrows():
        if counter > 0:
            break #for debugging purposes
        #print( item[1][1]) #item[1][1] is the productId
        #print( item[1][1], " ", getItemInfoVector(item[1][1]))
        #print( item[1][2]) #should be the item rating
        print("\n\n\nFinding similar item for:\n", item)
        result = getMostSimilarItem( item , owned)
        #print( "\nFound similar item!!!")
        #print( result, "\tUser's og item rating", item[1][2])
        newResult = (result[0]*item[1][2], result[1])
        print( newResult, "\tis the new result tuple (taking into account the rating of ", item[1][2], ")\n")
        resultsList.append( newResult)
        counter+=1
    resultsList.sort(key=lambda tup: tup[0], reverse=True)
    return resultsList

print( "done")

done


In [78]:
recommendItems("A2UDD2XDPD80BH" )

splitting data
~~~~~ 13  reviews~~~~~  split into:  7  and  6 ~~~~~
reviews sorted by time: 
             reviewerId   productId  rating        time
228327  A2UDD2XDPD80BH  B00067AVT2     5.0  1142208000
204696  A2UDD2XDPD80BH  B00020LZAW     5.0  1142553600
270903  A2UDD2XDPD80BH  B000B43OXU     5.0  1144540800
198242  A2UDD2XDPD80BH  B0001I1LD6     5.0  1145664000
304679  A2UDD2XDPD80BH  B000FBMITA     5.0  1149206400
289093  A2UDD2XDPD80BH  B000C427NM     5.0  1155427200
285552  A2UDD2XDPD80BH  B000BLM5PG     5.0  1156809600
258699  A2UDD2XDPD80BH  B0009WPZM2     5.0  1159142400
237315  A2UDD2XDPD80BH  B0007PLKZM     5.0  1160092800
124881  A2UDD2XDPD80BH  B000066TS5     1.0  1160956800
284015  A2UDD2XDPD80BH  B000BI3QK2     5.0  1161648000
319037  A2UDD2XDPD80BH  B000FRS9II     5.0  1167436800
289669  A2UDD2XDPD80BH  B000CBVC0Y     5.0  1170979200
-----------
Data used for recommedations:
             reviewerId   productId  rating        time
228327  A2UDD2XDPD80BH  B00067AVT2    

[(0.996768169496795, 'B00005TNI6')]